In [1]:
#!pip install argparse
#!pip3 install pytesseract

In [2]:
#importing libraries
import cv2
import argparse
import numpy as np
import pytesseract as py
import pandas as pd
#from utils import logger as logger

In [3]:
'''
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True,
                help = 'path to input image')
args = ap.parse_args()
'''

"\nap = argparse.ArgumentParser()\nap.add_argument('-i', '--image', required=True,\n                help = 'path to input image')\nargs = ap.parse_args()\n"

In [4]:
#Using YOLO-V3 

def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

 
#reading the image file
image = cv2.imread('images/repo5.jpg')

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

classes = None

with open('yolov3-classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv2.dnn.readNet('yolov3.weights', 'cfg/yolov3-custom.cfg')

blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

net.setInput(blob)

outs = net.forward(get_output_layers(net))

class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


for out in outs:
    for detection in out:
        scores = detection[:5]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([round(x), round(y), round(w), round(h)])


indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

In [5]:
indices

array([107, 360, 378, 345, 354, 369, 347, 353, 371, 377, 365, 356, 374,
       368, 380, 350, 362, 344,  18, 359,  36, 346,  57,  76,  97, 146,
       121, 341,  35,  17, 302, 263, 185, 224, 120,  56, 343, 145,  96,
        75, 340, 349, 262, 301,  16,  95, 261, 119,  55, 144, 339, 321,
       327, 312, 303,  83, 308,  33,  15, 338, 311, 314,  32, 323, 335,
       332, 317,  53, 329, 326, 299,  54,  72, 320,  94,  73,  93, 117,
       305, 142, 143, 260, 118,  14, 221, 220, 310, 337, 182, 304, 259,
       298, 334,  31,  71, 219, 249, 132, 285, 294, 267, 269, 268, 272,
       275,  30, 293, 296,  12, 278, 290, 284, 266,  29, 140,  91, 115,
        51,  90,  70, 257, 265, 179, 218,  11, 177, 243, 237,  43, 225,
        99, 230, 233, 229, 236, 227,   9, 239,  27, 254,  26, 245,  88,
       137, 112,  48,  67, 215, 176,   7,  59, 204, 195, 191, 194, 190,
         6, 188, 197, 200,  24,  45,  23, 134, 206,   5,  22, 168, 150,
       159, 124,  78,   0, 152, 149,   3, 155,  21, 151, 131,  4

In [6]:
image.shape

(635, 1068, 3)

In [7]:

#----------------------------OCR-------------------------------------------------------------------------



#cropping detected objects and passing into tesseract OCR
A = []
for i in indices:
    box = boxes[i]
    #print(str(i), box[0], box[1], box[2], box[3])
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    crop_img = image[y:y+h, x:x+w]
    if crop_img.shape[0] > 0 and crop_img.shape[1] > 0:
#         print(crop_img.shape)
#         cv2.imshow('crop',image)
        cv2.imwrite("Crop/crop__" + str(i) + ".jpg",crop_img)
        # grayscale region within bounding box
        gray = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
        # resize image to three times as large as original for better readability
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        # perform gaussian blur to smoothen image
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        # threshold the image using Otsus method to preprocess for tesseract
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
        #perfrom bitwise not to flip image to black text on white background
        roi = cv2.bitwise_not(thresh)
        #saving the roi regions
        cv2.imwrite("Crop/roi__" + str(i) + ".jpg",roi)
        #passing to tesseract
        c = py.image_to_string(roi, config = '--oem 3')
        A.append(c)

In [8]:
len(A)

163

In [9]:
A

['BY : SELF (7824357519), KALPANA MEDIC!\nLAB, BIHUTOLI ROAD ,HOJAL ASSAM\n"ASKED —s: AAROGYAM C ™\n\n',
 'LU ote Er to”\n\nPHOTOMETRY\n\nnical conditions.\n',
 '',
 'Or ih wh TREO\n\nA ALB/GLOBULIN RATIO\nf GLOBULIN __\n\ne correlate with clinical conditions.\n\nCA\n\nPi\n',
 'CALCULATED\nPHOTOMETRY\n\n',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'i wine thyrocare.com\n\nDAT :\n\\NA MEDICOS AND CITY\n,/HOJAI ,ASSAM 782435,782435\n',
 'ma/di\nma/dl\nU/|\nU/!\nU/|\ngm/di\ngm/dl\n\n<O.%\n0-0.9\n< 55\n\n< 35\n\n< 45\n5.7-8.2\n3.2-4.8\n',
 'CTED AT :\nALPANA MEDICOS AND CITY\nOAD ,HOJAI ,ASSAM 782435,782435\n\nLLiaTrTre BImmbae Ai PARIeeE\n',
 'nyrocare’\n\nink Thyroid. Think Thyrocare.\n\nmaa: - 400703\n“ TNyrocare.com\n\n',
 'UE =—sUNITS.—_—s NORMAL RANGE\nU/L 45 - 129\nmaq/dl 0.3-1.2\n',
 'UsI\ngm/di\ngm/dl\nRatio\nqm/dt\n\n< 49\n\n5.7-8.2\n3.2-4.8\n0.9-2\n2.5

In [10]:
#Splitting elements of A

B =[]
N = len(A)
for i in range(5):
    a = A[i].split('\n')
    print(a)
    B.append(a)

print(B)


# Conv into dictionary

#declaring the dict key values
K = ["Test Name", "Unit", "Reference Value", "Value"]
Data = dict(zip(K,B))
#print(Data[B])

#Creating a Dataframe
Y = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in Data.items() ]))

#print(Y.columns)
#Exporting to CSV 
Y.to_csv (r'out_csv/cust_ocr_new.csv', index = False, header=['Test Name', 'Unit', 'Reference Value', 'Value'])


#print(Y)

#------------------------for validating crop images---------------------
'''
img = cv2.imread('Crop/crop__1.jpg')

b = py.image_to_string(img,config='--oem 3')

# grayscale region within bounding box
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # resize image to three times as large as original for better readability
gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    # perform gaussian blur to smoothen image
blur = cv2.GaussianBlur(gray, (5,5), 0)
    #cv2.imshow("Gray", gray)
    #cv2.waitKey(0)
    # threshold the image using Otsus method to preprocess for tesseract
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
cv2.imshow("Otsu Threshold", roi)
cv2.waitKey(0)

        # perfrom bitwise not to flip image to black text on white background
roi = cv2.bitwise_not(thresh)
        # perform another blur on character region

    
           
text = py.image_to_string(roi, config='--oem 3')
'''

['BY : SELF (7824357519), KALPANA MEDIC!', 'LAB, BIHUTOLI ROAD ,HOJAL ASSAM', '"ASKED —s: AAROGYAM C ™', '', '']
['LU ote Er to”', '', 'PHOTOMETRY', '', 'nical conditions.', '']
['']
['Or ih wh TREO', '', 'A ALB/GLOBULIN RATIO', 'f GLOBULIN __', '', 'e correlate with clinical conditions.', '', 'CA', '', 'Pi', '']
['CALCULATED', 'PHOTOMETRY', '', '']
[['BY : SELF (7824357519), KALPANA MEDIC!', 'LAB, BIHUTOLI ROAD ,HOJAL ASSAM', '"ASKED —s: AAROGYAM C ™', '', ''], ['LU ote Er to”', '', 'PHOTOMETRY', '', 'nical conditions.', ''], [''], ['Or ih wh TREO', '', 'A ALB/GLOBULIN RATIO', 'f GLOBULIN __', '', 'e correlate with clinical conditions.', '', 'CA', '', 'Pi', ''], ['CALCULATED', 'PHOTOMETRY', '', '']]


'\nimg = cv2.imread(\'Crop/crop__1.jpg\')\n\nb = py.image_to_string(img,config=\'--oem 3\')\n\n# grayscale region within bounding box\ngray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)\n    # resize image to three times as large as original for better readability\ngray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)\n    # perform gaussian blur to smoothen image\nblur = cv2.GaussianBlur(gray, (5,5), 0)\n    #cv2.imshow("Gray", gray)\n    #cv2.waitKey(0)\n    # threshold the image using Otsus method to preprocess for tesseract\nret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)\ncv2.imshow("Otsu Threshold", roi)\ncv2.waitKey(0)\n\n        # perfrom bitwise not to flip image to black text on white background\nroi = cv2.bitwise_not(thresh)\n        # perform another blur on character region\n\n    \n           \ntext = py.image_to_string(roi, config=\'--oem 3\')\n'

In [11]:
Y.shape

(11, 4)

In [12]:
Data.items()

dict_items([('Test Name', ['BY : SELF (7824357519), KALPANA MEDIC!', 'LAB, BIHUTOLI ROAD ,HOJAL ASSAM', '"ASKED —s: AAROGYAM C ™', '', '']), ('Unit', ['LU ote Er to”', '', 'PHOTOMETRY', '', 'nical conditions.', '']), ('Reference Value', ['']), ('Value', ['Or ih wh TREO', '', 'A ALB/GLOBULIN RATIO', 'f GLOBULIN __', '', 'e correlate with clinical conditions.', '', 'CA', '', 'Pi', ''])])